# Graph Coloring Problem

La presente implementación detalla en primera instancia la solución del problema de coloración de vertices en grafos mediante programación lineal entera. Sobre esta se generan distintas instancias del problema en base a los sets dispuestos por https://mat.tepper.cmu.edu/COLOR/instances.html, no se utilizaron la totalidad de los sets dispuestos debido a la alta cantidad de vertices que contenian parte de estos, lo cual provocaba una escalada en tiempos de ejecución muy elevada.

In [ ]:
import Pkg
using JuMP
using GLPK
using Glob
using IJulia

## Función de evaluación graph_coloring
La siguiente función detalla el procedimiento de coloración mediante backtracking.

En esta se representan como variables de decición binarias sobre $g$ como, la cantidad de vertices $i\in(1,...,n)$ por los colores $c\in(1,...,\Delta(g))$, donde se tendrian las variables $x_{ij}$ con $n*\Delta(g)$, donde $\Delta(g)$ corresponde al grado maximo del grafo y es una cota superior que permite reducir el valor de $num_{colors}$ 

Las restricciones son las siguientes.
- Los vertices adyacentes al vertice v no pueden tener el mismo color c, tal que la suma de los $c$ utilizados por los $x_{v,c}$ adyacentes sea igual a 1 (unico color en el vecindario).
- La cantidad de colores utilizados no puede exceder la cantidad de vertices en el grafo.

Como función objetivo se tiene que minimizar la cantidad de vertices coloreados de distinta manera, representados mediante la suma las variables $x$

In [ ]:
function graph_coloring(graph)
    num_nodes = length(graph)
    max_degree = 0
    for i in 1:num_nodes
        degree = sum(graph[i])
        if degree > max_degree
            max_degree = degree
        end
    end
    println("max_degree: ", max_degree)

    # Crear el modelo de programación lineal
    model = Model(GLPK.Optimizer)

    # Variables de decisión: x[i, j] indica si el nodo i tiene el color j
    @variable(model, x[1:num_nodes, 1:max_degree], Bin)

    # Restricción: cada nodo debe tener exactamente un color asignado
    for i in 1:num_nodes
        @constraint(model, sum(x[i, :]) == 1)
    end

    # Restricción: dos nodos adyacentes no pueden tener el mismo color
    for i in 1:num_nodes
        for j in 1:num_nodes
            if graph[i][j] == 1
                for k in 1:max_degree
                    @constraint(model, x[i, k] + x[j, k] <= 1)
                end
            end
        end
    end

    # Función objetivo: minimizar el número de colores utilizados
    @objective(model, Min, sum(x))

    # Resolver el modelo
    optimize!(model)

    # Obtener los resultados
    coloring = Dict()
    for i in 1:num_nodes
        for j in 1:max_degree
            if value(x[i, j]) == 1
                coloring[i] = j
                break
            end
        end
    end

    return maximum(values(coloring))
end

## Clase GraphMinified

Utilizada para la lectura de ficheros, almacena el nombre del archivo, la cantidad de vertices y las aristas entre cada vertice como una lista de aristas.

In [ ]:
struct GraphMinified
    name::AbstractString
    vertices::Int
    edges::Vector{Tuple{Int, Int}}
end

In [ ]:
file_names = glob("./instancias/*.col")

graphs_files = []
for file_name in file_names
    open(file_name) do file
        lines = readlines(file)
        vertices = 0
        edges = []
        name = file_name[14:end]
        for line in lines
            if startswith(line, "p")
                vertices = parse(Int, split(line)[3])
            elseif startswith(line, "e")
                push!(edges, (parse(Int, split(line)[2]), parse(Int, split(line)[3])))
            end
        end
        graph = GraphMinified(name, vertices, edges)
        push!(graphs_files, graph)
    end
end

# ordenar por número de vértices
sort!(graphs_files, by = x -> x.vertices)

println("Número de grafos: ", length(graphs_files))
for graph in graphs_files
    println("Nombre: ", graph.name, " Número de vértices: ", graph.vertices, " Número de aristas: ", length(graph.edges))
end

In [5]:
# itera por cada instancia
results = []

for graph_file in graphs_files
    println("Trabajando Grafo: ", graph_file.name, " Número de vértices: ", graph_file.vertices, " Número de aristas: ", length(graph_file.edges))
    elapsed_time_data = Float64[]
    num_colors_data = Float64[]
    mem_size_data = Float64[]
    for i in 1:1
        graph = []
        for j in 1:graph_file.vertices
            push!(graph, zeros(Int, graph_file.vertices))
        end
        for edge in graph_file.edges
            graph[edge[1]][edge[2]] = 1
            graph[edge[2]][edge[1]] = 1
        end
        mem_size = Base.summarysize(graph)
        elapsed_time = @elapsed num_colors = graph_coloring(graph)
        push!(elapsed_time_data, elapsed_time)
        push!(num_colors_data, num_colors)
        push!(mem_size_data, mem_size)
    end
    # determina la media de los resultados
    elapsed_time = sum(elapsed_time_data) / length(elapsed_time_data)
    num_colors = sum(num_colors_data) / length(num_colors_data)
    mem_size = sum(mem_size_data) / length(mem_size_data)
    res = [graph_file.name, graph_file.vertices, length(graph_file.edges), elapsed_time, num_colors, mem_size]
    push!(results, res)
    println("Resultados: ", res)
end

Trabajando Grafo: myciel3.col Número de vértices: 11 Número de aristas: 20
Resultados: 

Any[

"myciel3.col", 11, 20, 

2.9985061, 4.0, 1536.0]
Trabajando Grafo: myciel4.col Número de vértices: 23 Número de aristas: 71


Resultados: Any["myciel4.col", 23, 71, 0.0947831, 5.0, 5376.0]
Trabajando Grafo: queen5_5.col Número de vértices: 25 Número de aristas: 320

...

, 12136.0]
Trabajando Grafo: myciel5.col Número de vértices: 47 Número de aristas: 236


Resultados: Any["myciel5.col", 47, 236, 2.6339832, 6.0, 19968.0]
Trabajando Grafo: queen7_7.col Número de vértices: 49 Número de aristas: 952


Resultados: Any["queen7_7.col", 49, 952, 10.0872436, 10.0, 21600.0]
Trabajando Grafo: queen8_8.col Número de vértices: 64 Número de aristas: 1456


Resultados: Any["queen8_8.col", 64, 1456, 41.2400405, 13.0, 35880.0]
Trabajando Grafo: huck.col Número de vértices: 74 Número de aristas: 602


Resultados: Any["huck.col", 74, 602, 22.1901015, 11.0, 47400.0]
Trabajando Grafo: jean.col Número de vértices: 80 Número de aristas: 508
